##Regional Aid Density (by Area)


In [2]:

!pip install pyshp
!pip install pyproj
!pip install geopy

!sudo apt install libspatialindex-dev
!pip install rtree
!pip install pygeos
!pip install folium
!pip install geopandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 3.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.8 MB 24.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 7 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space

In [3]:

import pygeos
from shapely.geometry import Point
import geopandas as gpd
import os
import pandas as pd
import numpy as np

import shapefile
import pyproj #https://stackoverflow.com/questions/68317672/coordinate-conversion-script-isnt-giving-me-an-accurate-reading-svy21-to-wgs84
import geopy.distance #https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

import geopandas as gpd
import folium
import branca
import branca.colormap as cm
from google.colab import data_table

data_table.enable_dataframe_formatter()

Loading in datasets directly from sites.

1.   Regional dataset 

link:https://geoportal.icpac.net/layers/geonode%3Asom_admbnda_adm2_undp

2. S1 from event breif






In [8]:
#Turning Somalia Regional data into Geo_Dataframe
Region_shp= gpd.read_file('https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Asom_admbnda_adm2_undp&outputFormat=SHAPE-ZIP&srs=EPSG%3A4326&format_options=charset%3AUTF-8')
Region_shp.geometry=Region_shp.geometry.to_crs(epsg='4326')

#Kenya_K1_dataset (to observe)
Ken_AidCoord=pd.read_excel("https://data.humdata.org/dataset/cda96bea-ee01-41ea-b5ef-6e455c770557/resource/4fa5ea69-7020-494e-8245-4771d2f6b7d0/download/kenya-3w-final-list-2017.xlsx",header=0)

#Somalia_S1_dataset (to observe)
Som_AidCoord=pd.read_excel("https://data.humdata.org/dataset/8dfc1407-8165-44f2-93c3-f294172ce6c6/resource/10a1749b-1484-496e-b505-b6a34a40ec4a/download/ocha_som_operational_presence_3w_data_oct2022.xlsx",header=0,skiprows=[1])


Checking for errors
Data cleaning
Google sheets detected some errors, so i am doing a formal test here.

In [10]:

#########
###Checking column names
print("Before renaming:",list(Som_AidCoord)) #some oddly named columns observed
Som_AidCoord.rename(columns = {'Cluster ':'Cluster', 'Org. Name':'Org_Name','Org. Acronym':'Org_Acronym','Org Type':'Org_Type',"Region ":"Region","District ":"District"}, inplace = True)
print("After renaming:",list(Som_AidCoord)) #checking new columns

##########
###Checking for Differences in naming convention of Districts
print('Before cleaning data:',sorted(list(Som_AidCoord['District'].unique())))

#errors found: Berbera,LUUQ(alphabetical),Xudun,Caynabo,Dinnsoor,Iskushuban,Jowhar,xudun
#Naming Convention: Only First letter will be Cap, no spaces after
Som_AidCoord['District'].replace(['Berbera ', 'LUUQ', 'xudun', 'caynabo','diinsoor','eyl','iskushuban','jowhar'], ['Berbera', 'Luuq', 'Xudun', 'Cayanabo','Dinnsoor','Eyl','Iskushuban','Jowhar'],inplace=True)
print('After cleaning data:',sorted(list(Som_AidCoord['District'].unique())))

##############
###Checking for Differences in naming convention of Districts
print('Before cleaning data:',sorted(list(Som_AidCoord['Org_Acronym'].unique())))
#print(Som_AidCoord.loc[Som_AidCoord['Org_Acronym']=='KRD\xa0'])
Som_AidCoord['Org_Acronym'].replace([' YOVENCO','KRD\xa0','LLG\xa0'], ['YOVENCO','KRD','LLG'],inplace=True)
print('After cleaning data:',sorted(list(Som_AidCoord['Org_Acronym'].unique())))

####################
###realised while doing data manipulation that there were inconsistencies in naming districts between 2 datasets.
print(sorted(list(Region_shp['admin2Name'].unique())))
Region_shp['admin2Name']=Region_shp['admin2Name'].str.strip()


print(sorted(list(Som_AidCoord['District'].unique())))
Som_AidCoord['District']=Som_AidCoord['District'].str.strip()

#Get dataframe list of names of both districts
Region_name=pd.DataFrame(list(Region_shp['admin2Name'].unique()))
District_name=pd.DataFrame(list(Som_AidCoord['District'].unique()))

#merge datasets to see which ones doesn't match
Merged_test=pd.merge(Region_name,District_name,on=0,how='outer',indicator=True)
#print(Merged_test.loc[(Merged_test['_merge']!='both')])

#Changing names on datasets

Som_AidCoord['District'].replace(['Dollow','Wanla Weyne','Bardheere','Bu’aale','Dinnsoor','Cayanabo'], ['Doolow','Wanla Weyn','Baardheere',"Bu'aale",'Diinsoor','Caynabo'],inplace=True)


##Test again by merge:

#Get dataframe list of names of both districts
Region_name=pd.DataFrame(list(Region_shp['admin2Name'].unique()))
District_name=pd.DataFrame(list(Som_AidCoord['District'].unique()))

#merge datasets to see which ones doesn't match
Merged_test=pd.merge(Region_name,District_name,on=0,how='outer',indicator=True)
print(Merged_test.loc[(Merged_test['_merge']!='both')])

Merged_test

Before renaming: ['Cluster', 'Org_Name', 'Org_Acronym', 'Org_Type', 'Region', 'District']
After renaming: ['Cluster', 'Org_Name', 'Org_Acronym', 'Org_Type', 'Region', 'District']
Before cleaning data: ['Adan Yabaal', 'Afgooye', 'Afmadow', 'Baardheere', 'Badhaadhe', 'Baki', 'Balcad', 'Banadir', 'Bandarbeyla', 'Baraawe', 'Baydhaba', 'Belet Weyne', 'Belet Xaawo', 'Berbera', 'Borama', 'Bossaso', "Bu'aale", 'Bulo Burto', 'Burco', 'Burtinle', 'Buuhoodle', 'Buur Hakaba', 'Cabudwaaq', 'Cadaado', 'Cadale', 'Caluula', 'Caynabo', 'Ceel Afweyn', 'Ceel Barde', 'Ceel Buur', 'Ceel Dheer', 'Ceel Waaq', 'Ceerigaabo', 'Dhuusamarreeb', 'Diinsoor', 'Doolow', 'Eyl', 'Gaalkacyo', 'Galdogob', 'Garbahaarey', 'Garoowe', 'Gebiley', 'Hargeysa', 'Hobyo', 'Iskushuban', 'Jalalaqsi', 'Jamaame', 'Jariiban', 'Jilib', 'Jowhar', 'Kismaayo', 'Kurtunwaarey', 'Laas Caanood', 'Laasqoray', 'Lughaye', 'Luuq', 'Marka', 'Owdweyne', 'Qandala', 'Qansax Dheere', 'Qardho', 'Qoryooley', 'Rab Dhuure', 'Saakow', 'Sheikh', 'Taleex', 'T

0 _merge
0     Banadir   both
1      Borama   both
2        Baki   both
3     Lughaye   both
4      Zeylac   both
..        ...    ...
69     Saakow   both
70   Kismaayo   both
71    Afmadow   both
72  Badhaadhe   both
73    Jamaame   both

[74 rows x 2 columns]

Data manipulation

In [12]:
#Get a count of operational presence by region
Somalia_Ops_presence=Som_AidCoord[['District','Org_Acronym']].groupby(['District']).count()
#change coordinate reference system and get area
Region_shp.geometry=Region_shp.geometry.to_crs(epsg='4326')
Region_shp['Area']=Region_shp.geometry.area

#Aid provided by Region
Som_Region_OPs=pd.merge(Somalia_Ops_presence,Region_shp,left_on='District',right_on='admin2Name',how='right',indicator=True)
Som_Region_OPs = gpd.GeoDataFrame(Som_Region_OPs, geometry='geometry')
#Operation presence fillna with 0
Som_Region_OPs['Org_Acronym']=Som_Region_OPs['Org_Acronym'].fillna(0.0001)

#get Regional operational presence density
Som_Region_OPs['Ops_Density']=Som_Region_OPs.apply(lambda x: x.Org_Acronym/x.Area,axis=1)

<ipython-input-12-8f86b78fc544>:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  Region_shp['Area']=Region_shp.geometry.area


Mapping Choropleth

In [13]:
Somalia=[8.408416,48.4828]
OP=folium.Map(location=Somalia, zoom_start=6)



#guide:https://towardsdatascience.com/the-battle-of-choropleths-part-3-folium-86ab1232afc
###START OF CONFUSION
#Admittedly i am rather unfamiliar with the more in-depth tinkering of Folium, however i got what i needed to work, to work.
#
 
vmax = Som_Region_OPs['Ops_Density'].max()
vmin = Som_Region_OPs['Ops_Density'].min()
colormap = cm.LinearColormap(colors=['red','orange','yellow', '#c7f6b6' ,'#c7f6b6','green'], 
                             vmin=vmin,
                             vmax=vmax,caption='Regional Operational Presence in Somalia')

n_steps = 6 # Quantiles
list_of_values = Som_Region_OPs['Ops_Density'].sort_values()
#Remove all 0's as the geopandas version did not count 0 on the count of merging
list_of_values = [i for i in list_of_values if i != 0]
length = len(list_of_values)
index = [list_of_values[int((length/n_steps)*i)] for i in range(n_steps)]
print(index)
colormap = colormap.to_step(index=index)
colormap.add_to(OP)
#Add the colormap as a legend


# First create a function that would call on these values
style_function = lambda x: {"weight":1.0, 
                            'color':'brown',
                            'fillColor':colormap(x['properties']['Ops_Density']), 
                            'fillOpacity':0.3}

#########END OF CONFUSION#########


#Save it as an object so you can add it


for _, r in Region_shp.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=Som_Region_OPs,style_function=style_function,
                           overlay=True)
    geo_j.add_to(OP)  





display(OP)
OP.save('/content/Somalia_Ops_Presence.html')

Output hidden; open in https://colab.research.google.com to view.

Org_Acronym  OBJECTID_1 admin2Name admin2Pcod admin2RefN admin2AltN  \
0        3033.0           1    Banadir     SO2201    Banadir       None   
1         219.0           2     Borama     SO1101     Borama    Boorama   
2         260.0           3       Baki     SO1102       Baki       None   
3         106.0           4    Lughaye     SO1103    Lughaye       None   
4          79.0           5     Zeylac     SO1104     Zeylac     Saylac   
..          ...         ...        ...        ...        ...        ...   
69        430.0          70     Saakow     SO2703     Saakow      Sakow   
70       1451.0          71   Kismaayo     SO2801   Kismaayo    Kismayo   
71       1043.0          72    Afmadow     SO2802    Afmadow       None   
72        481.0          73  Badhaadhe     SO2803  Badhaadhe   Badhadhe   
73        168.0          74    Jamaame     SO2804    Jamaame     Jamame   

   admin2Al_1   admin1Name admin1Pcod admin0Name admin0Pcod        date  \
0        None      Banadir       SO22    Somalia         SO  2014-06-06   
1        None        Awdal       SO11    Somalia         SO  2014-06-06   
2        None        Awdal       SO11    Somalia         SO  2014-06-06   
3        None        Awdal       SO11    Somalia         SO  2014-06-06   
4        None        Awdal       SO11    Somalia         SO  2014-06-06   
..        ...          ...        ...        ...        ...         ...   
69       None  Middle Juba       SO27    Somalia         SO  2014-06-06   
70       None   Lower Juba       SO28    Somalia         SO  2014-06-06   
71       None   Lower Juba       SO28    Somalia         SO  2014-06-06   
72       None   Lower Juba       SO28    Somalia         SO  2014-06-06   
73       None   Lower Juba       SO28    Somalia         SO  2014-06-06   

       validOn ValidTo  Shape_Leng  Shape_Area  \
0   2017-04-05    None    0.864205    0.019619   
1   2017-04-05    None    2.403698    0.254798   
2   2017-04-05    None    2.871573    0.282360   
3   2017-04-05    None    2.334272    0.275737   
4   2017-04-05    None    3.852047    0.499583   
..         ...     ...         ...         ...   
69  2017-04-05    None    3.868911    0.654011   
70  2017-04-05    None    5.532878    0.749813   
71  2017-04-05    None    5.871596    2.174108   
72  2017-04-05    None    4.657717    0.793048   
73  2017-04-05    None    1.825811    0.175052   

                                             geometry      Area _merge  \
0   POLYGON ((45.61720 2.18585, 45.61871 2.18601, ...  0.019619   both   
1   POLYGON ((43.35094 10.05094, 43.33094 9.84096,...  0.254798   both   
2   POLYGON ((44.23660 10.47312, 44.15096 10.32091...  0.282360   both   
3   POLYGON ((44.07241 10.59786, 43.46094 10.38091...  0.275737   both   
4   MULTIPOLYGON (((43.46797 11.42739, 43.45518 11...  0.499583   both   
..                                                ...       ...    ...   
69  POLYGON ((42.95330 1.48243, 42.23090 1.51171, ...  0.654011   both   
70  MULTIPOLYGON (((42.03180 -1.01986, 42.02406 -1...  0.749813   both   
71  POLYGON ((41.00146 -0.19648, 40.99565 -0.19731...  2.174108   both   
72  MULTIPOLYGON (((41.92670 -1.16192, 41.92260 -1...  0.793048   both   
73  POLYGON ((42.72680 0.32868, 42.73091 0.31182, ...  0.175052   both   

    Ops_Density  
0      0.000006  
1      0.001163  
2      0.001086  
3      0.002601  
4      0.006324  
..          ...  
69     0.001521  
70     0.000517  
71     0.002084  
72     0.001649  
73     0.001042  

[74 rows x 20 columns]